In [12]:
data Ty
  = Int         -- int
  | Arr Ty Int  -- ty[3]
  | Ptr Ty      -- ty*
  | Str String  -- struct list
  deriving Show

data SDec = SDec String [VDec]     -- struct list { ... };
  deriving Show

data VDec = VDec Ty String         -- int x;
  deriving Show

data FDec = FDec Ty String [VDec] [VDec] Stmt
  deriving Show

data Stmt
  = Sexp Exp                  --  e;
  | Sseq [Stmt]               --  { ss }  where ss is either empty or e1; e2; ... 
  | If Exp Stmt               --  if (e) s
  | IfElse Exp Stmt Stmt      --  if (e) s1 else s2
  -- | Switch Exp [Stmt] Stmt    --  switch (e) { case 0: ss1 break; case 1: ss0 break; ... default: ssk }
  | While Exp Stmt            --  while (e) s
  | For (Exp, Exp, Exp) Stmt  --  for (e1; e2; e3) s
  deriving Show
  
data Exp 
  = Lit Int            --  3
  | Var String         --  x
  | And Exp Exp        --  e1 && e2
  | Or Exp Exp         --  e1 || e2
  | Not Exp            --  ! e
  | Neg Exp            --  - e
  | Add Exp Exp        --  e1 + e2
  | Sub Exp Exp        --  e1 - e2
  | Mul Exp Exp        --  e1 * e2
  | Div Exp Exp        --  e1 / e2
  | Assign Exp Exp     --  e1 <- e2
  | Sizeof Ty          --  sizeof(ty)
  | Ref                --  & e
  | Deref              --  * e
  | Func String [Exp]  -- f(e1, e2, ...)
  deriving Show

data Prog = Prog [SDec] [VDec] [FDec] Stmt  deriving Show

In [13]:
import qualified Data.Map as Map
import Data.Map (Map, (!)) -- Data.Map 모듈에서 Map 타입과 !연산자만

type Addr = Int
type Mem = Map Addr Int

emptyMem :: Mem
emptyMem = Map.empty

In [14]:
:type Map.insert

Map.insert 10 3 emptyMem
Map.insert 11 5 . Map.insert 10 3 $ emptyMem
Map.insert 10 4 . Map.insert 11 5 . Map.insert 10 3 $ emptyMem

mem3 = Map.insert 10 4 . Map.insert 11 5 . Map.insert 10 3 $ emptyMem

Map.insert :: forall k a. Ord k => k -> a -> Map k a -> Map k a

fromList [(10,3)]

fromList [(10,3),(11,5)]

fromList [(10,4),(11,5)]

In [15]:
:type Map.lookup

Map.lookup 10 mem3
Map.lookup 11 mem3
Map.lookup 12 mem3

Map.lookup :: forall k a. Ord k => k -> Map k a -> Maybe a

Just 4

Just 5

Nothing

In [16]:
:type (!)

mem3 ! 10
mem3 ! 11
mem3 ! 12

(!) :: forall k a. Ord k => Map k a -> k -> a

4

5

Map.!: given key is not an element in the map
CallStack (from HasCallStack):
  error, called at libraries/containers/Data/Map/Internal.hs:603:17 in containers-0.5.10.2:Data.Map.Internal

In [17]:
import Data.Maybe (fromJust)

:type fromJust

fromJust (Just 4)

fromJust Nothing

fromJust :: forall a. Maybe a -> a

4

Maybe.fromJust: Nothing

In [18]:
lookup' x = fromJust . lookup x

:type lookup
:type lookup'

lookup :: forall a b. Eq a => a -> [(a, b)] -> Maybe b

lookup' :: forall a c. Eq a => a -> [(a, c)] -> c

In [31]:
type Code = [Inst]

data Inst
  = ADD | SUB | MUL | DIV | NEG
  | AND | OR | NOT
  | EQU | NEQ | GR | GEQ | LE | LEQ
  | DUP | POP
  | JUMP Code | JUMPz Code   --  | JUMPi [Code]
  | LOAD | LOADc Int    --  | LOADa | LOADr | LOADrc
  | STORE Addr | STOREc Addr Int  --  | STOREa | STOREr
--  | ALLOC | SLIDE | CALL | ENTER  | RETURN
--  | MALLOC | NEW
--  | MARK
  | HALT
  deriving Show
  
type AEnv = [(String, Addr)]
-- type TEnv = [(String, Ty)]
-- type SEnv = [(String, [TEnv])]

In [32]:
type Stack = [Int]
type VM = (Stack, Mem)

b2i False = 0
b2i True  = 1

i2b = (/= 0)

runVM :: Code -> VM -> VM
runVM [] vm = vm
runVM (HALT:_) vm = vm
runVM (JUMP c : code) vm = runVM c vm
runVM (JUMPz c : code) ([], _) = error $ "JUMPz on empty stack"
runVM (JUMPz c : code) (0:stack, mem) = runVM c (stack, mem)
runVM (JUMPz c : code) (_:stack, mem) = runVM code (stack, mem)
runVM (inst : code) vm = runVM code (stepVM inst vm)

stepVM :: Inst -> VM -> VM
stepVM ADD (x:y:stack , mem) = (y + x : stack, mem)
stepVM SUB (x:y:stack , mem) = (y - x : stack, mem)
stepVM MUL (x:y:stack , mem) = (y * x : stack, mem)
stepVM DIV (x:y:stack , mem) = (y `div` x : stack, mem)
stepVM NEG (x:stack , mem) = (- x : stack , mem)
stepVM AND (x:y:stack , mem) = (b2i(i2b y && i2b x) : stack, mem)
stepVM OR  (x:y:stack , mem) = (b2i(i2b y || i2b x) : stack, mem)
stepVM NOT (x:stack , mem) = (b2i(not $ i2b x) : stack, mem)
stepVM EQU (x:y:stack , mem) = (b2i(x==y) : stack, mem)
stepVM NEQ (x:y:stack , mem) = (b2i(x/=y) : stack, mem)
stepVM GR  (x:y:stack , mem) = (b2i(y> x) : stack, mem)
stepVM GEQ (x:y:stack , mem) = (b2i(y>=x) : stack, mem)
stepVM LE  (x:y:stack , mem) = (b2i(y< x) : stack, mem)
stepVM LEQ (x:y:stack , mem) = (b2i(y<=x) : stack, mem)
stepVM DUP (x:stack , mem) = (x:x:stack, mem)
stepVM POP (x:stack , mem) = (stack, mem)
stepVM (LOAD) (addr:stack, mem) = (mem!addr : stack, mem)
stepVM (LOADc val) (stack, mem) = (val:stack, mem)
--  | LOADa | LOADr | LOADrc
stepVM (STORE  addr) (v:stack, mem) = (stack, Map.insert addr v mem)
stepVM (STOREc addr v) (stack, mem) = (stack, Map.insert addr v mem)
--  | STOREa | STOREr
--  | ALLOC | SLIDE | CALL | ENTER  | RETURN
--  | MALLOC | NEW
--  | MARK
stepVM inst@(JUMP _) _ = error $ "stepVM "++show inst
stepVM inst@(JUMPz _) _ = error $ "stepVM "++show inst
-- stepVM inst@(JUMPi _) _ = error $ "stepVM "++show inst
stepVM HALT _ = error "stepVM HALT ..."
stepVM inst vm = error $ "stemVM "++show inst++" "++show vm

In [33]:
codeR :: Exp -> AEnv -> Code
codeR (Lit q) rho = [LOADc q]
codeR (Var x) rho = codeL (Var x) rho ++ [LOAD]
codeR (And e1 e2) rho = codeR e1 rho ++ codeR e2 rho ++ [AND]
codeR (Or e1 e2)  rho = codeR e1 rho ++ codeR e2 rho ++ [OR]
codeR (Not e) rho = codeR e rho ++ [NOT]
codeR (Neg e) rho = undefined
codeR (Add e1 e2) rho = undefined
codeR (Sub e1 e2) rho = undefined
codeR (Mul e1 e2) rho = undefined
codeR (Div e1 e2) rho = undefined
codeR (Assign e1 e2) rho = undefined

codeL :: Exp -> AEnv -> Code
codeL (Var x) rho = [LOADc xAddr]  where xAddr = lookup' x rho

code :: Stmt -> AEnv -> Code
code (Sexp e)      rho = codeR e rho ++ [POP]
code (Sseq (s:ss)) rho = code s rho ++ code (Sseq ss) rho
code (Sseq [])     rho = []
code (If e s)     rho = undefined
code (IfElse e s1 s2) rho = undefined
-- code (Switch e ss d) = undefined
code (While e s) rho = undefined
code (For (e1,e2,e3) s) rho = undefined

:type codeL
:type codeR
:type code

codeL :: Exp -> AEnv -> Code

codeR :: Exp -> AEnv -> Code

code :: Stmt -> AEnv -> Code